In [2]:
import openai
import ipywidgets as widgets
from IPython.display import display, Markdown
import os
import json
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path="../.env")

# Toggle Azure OpenAI usage based on environment
USE_AZURE = os.getenv("USE_AZURE", "False").lower() == "true"

# API credentials and model config
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4")

AZURE_API_BASE = os.getenv("AZURE_API_BASE")
AZURE_API_KEY = os.getenv("AZURE_API_KEY")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION", "2023-03-15-preview")

# Configure OpenAI client
if USE_AZURE:
    openai.api_type = "azure"
    openai.api_base = AZURE_API_BASE
    openai.api_version = AZURE_API_VERSION
    openai.api_key = AZURE_API_KEY
else:
    openai.api_key = OPENAI_API_KEY

# Class to manage individual chat sessions
class ChatSession:
    def __init__(self, session_id=None):
        self.session_id = session_id or datetime.now().strftime("%Y%m%d_%H%M%S")
        self.history = [{"role": "system", "content": "You are ChatGPT, a helpful assistant."}]

    def add_user_message(self, content):
        self.history.append({"role": "user", "content": content})

    def add_assistant_message(self, content):
        self.history.append({"role": "assistant", "content": content})

    def save_to_file(self):
        # Persist session history locally
        os.makedirs("../chat_sessions", exist_ok=True)
        with open(f"../chat_sessions/{self.session_id}.json", "w") as f:
            json.dump(self.history, f, indent=2)

# Query OpenAI or Azure OpenAI model
def query_chatgpt(messages):
    if USE_AZURE:
        response = openai.ChatCompletion.create(
            engine=AZURE_DEPLOYMENT_NAME,
            messages=messages,
            temperature=0.7,
        )
    else:
        response = openai.ChatCompletion.create(
            model=OPENAI_MODEL,
            messages=messages,
            temperature=0.7,
        )
    return response["choices"][0]["message"]["content"]

# Set up chat UI components
output_box = widgets.Output()
input_box = widgets.Textarea(
    placeholder="Type your message here...",
    layout=widgets.Layout(width="100%", height="80px")
)
send_btn = widgets.Button(description="Send", button_style="primary")
new_chat_btn = widgets.Button(description="New Chat", button_style="warning")
session_label = widgets.Label()

# Initialize first chat session
current_session = ChatSession()
session_label.value = f"Current Session: {current_session.session_id}"

# Handle sending a message
def on_send(_):
    user_input = input_box.value.strip()
    if not user_input:
        return

    input_box.value = ""
    current_session.add_user_message(user_input)

    with output_box:
        display(Markdown(f"**You:** {user_input}"))

    try:
        response = query_chatgpt(current_session.history)
    except Exception as e:
        response = f"Error: {str(e)}"

    current_session.add_assistant_message(response)

    with output_box:
        display(Markdown(f"**GPT:** {response}"))

    current_session.save_to_file()

# Start a new chat session
def on_new_chat(_):
    global current_session
    current_session = ChatSession()
    session_label.value = f"Current Session: {current_session.session_id}"
    output_box.clear_output()
    input_box.value = ""

# Bind button actions
send_btn.on_click(on_send)
new_chat_btn.on_click(on_new_chat)

# Render the chat UI
ui = widgets.VBox([
    session_label,
    output_box,
    widgets.HBox([send_btn, new_chat_btn]),
    input_box
])

display(ui)


ModuleNotFoundError: No module named 'openai'